

---


# 00. About Assignment


---



**There are 1 question in this assignment**


**To complete this assignment:**

1.   Replace **all** `#TODO` comments with your answers.
2.   Run **all** code blocks under "01. Assignment" section.
3.   Run the code block under "02. Complete the Assignment" section.
4.   Confirm that you completed this assignment without cheating by typing `"I confirm"`
4.   A csv file named `w8_submission.csv ` and a h5 file named `model.h5` will appear on files (you can reach files on the bar on the left) after you run the block.
5. Download `w8_submission.csv`.
5.   Download `model.h5`.
5.   Send .csv and .h5 files to your instructors via Google Classroom.



---


# 01. Assignment


---



## Q0 - Personal Info

In [ ]:
from datetime import datetime

now = datetime.now()
datetimenow = now.strftime("%d/%m/%Y %H:%M:%S")
name = input("Enter your full name:  ")

Enter your full name:  Efe Kaan Güler


## Q1 - Creating training and evaluating a RNN

1.   Import libraries that you will use.
2.   Check https://www.tensorflow.org/datasets/catalog/imdb_reviews for documentation of the `imdb_reviews` dataset
2.   Load `imdb_reviews` dataset using `tensorflow_datasets` library.
4.   Preprocess the dataset. (Test data must be assigned to the variable `test_padded`)
6.   Create a RNN model and assign it to variable `model`. (do not use pre-trained models or embeddings)
7.   Compile the model. (Set `metrics = ["accuracy"]`)
8.   Train the model.
9.   Evaluate the model
11.   If you reached over %80 test accuracy, run the grading block
11.   To pass this assignment, the test accuracy must be over %80.







In [ ]:
# Import libraries
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds

In [ ]:
data, ds_info = tfds.load(
    "imdb_reviews",
    split=["train", "test"],
    with_info=True,
    shuffle_files=True,
    as_supervised=True,
)
train_data, test_data = data
ds_info.features

FeaturesDict({
    'label': ClassLabel(shape=(), dtype=int64, num_classes=2),
    'text': Text(shape=(), dtype=string),
})

In [ ]:
class_names = ds_info.features["label"].names
class_names

['neg', 'pos']

In [ ]:
sample = train_data.take(1)
for text, label in sample:
    print(text.numpy())
    print(label.numpy())

b"This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie's ridiculous storyline. This movie is an early nineties US propaganda piece. The most pathetic scenes were those when the Columbian rebels were making their cases for revolutions. Maria Conchita Alonso appeared phony, and her pseudo-love affair with Walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning. I am disappointed that there are movies like this, ruining actor's like Christopher Walken's good name. I could barely sit through it."
0


In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

train_texts = [x.numpy().decode('utf-8') for x, y in train_data]
test_texts = [x.numpy().decode('utf-8') for x, y in test_data]

tokenizer = Tokenizer(num_words=1000, oov_token="<OOV>")
tokenizer.fit_on_texts(train_texts)

vocab_size = len(tokenizer.word_index) + 1

train_sequences = tokenizer.texts_to_sequences(train_texts)
train_padded = pad_sequences(train_sequences, padding='post', maxlen=100)

test_sequences = tokenizer.texts_to_sequences(test_texts)
test_padded = pad_sequences(test_sequences, padding='post', maxlen=100)

print(f"Padded training sequences have shape: {train_padded.shape}\n")
print(f"Padded validation sequences have shape: {test_padded.shape}")

Padded training sequences have shape: (25000, 100)

Padded validation sequences have shape: (25000, 100)


In [ ]:
train_labels = np.array([y.numpy() for x, y in train_data])
test_labels = np.array([y.numpy() for x, y in test_data])

print(f"Padded training sequences have shape: {train_labels.shape}\n")
print(f"Padded validation sequences have shape: {test_labels.shape}")

Padded training sequences have shape: (25000,)

Padded validation sequences have shape: (25000,)


In [ ]:
model = tf.keras.Sequential([
    tf.keras.Input(shape=(100,)),
    tf.keras.layers.Embedding(1000, 32),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(20, return_sequences=True)),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(20)),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(32, activation="relu"),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(1, activation="sigmoid")
    ])

# A simplier DNN model will perform better on this task, however our goal is to learn how to use RNN's.

In [ ]:
model.compile(loss=tf.keras.losses.binary_crossentropy,
              optimizer=tf.keras.optimizers.Adam(),
              metrics=['accuracy'])

In [ ]:
model.fit(
    train_padded,
    train_labels,
    epochs=5,
    validation_data=(test_padded, test_labels)
)

Epoch 1/5
782/782 ━━━━━━━━━━━━━━━━━━━━ 28s 27ms/step - accuracy: 0.6603 - loss: 0.5797 - val_accuracy: 0.8014 - val_loss: 0.4183
Epoch 2/5
782/782 ━━━━━━━━━━━━━━━━━━━━ 41s 32ms/step - accuracy: 0.8373 - loss: 0.3761 - val_accuracy: 0.8282 - val_loss: 0.3763
Epoch 3/5
782/782 ━━━━━━━━━━━━━━━━━━━━ 36s 26ms/step - accuracy: 0.8522 - loss: 0.3432 - val_accuracy: 0.8402 - val_loss: 0.3536
Epoch 4/5
782/782 ━━━━━━━━━━━━━━━━━━━━ 25s 33ms/step - accuracy: 0.8625 - loss: 0.3199 - val_accuracy: 0.8382 - val_loss: 0.3604
Epoch 5/5
782/782 ━━━━━━━━━━━━━━━━━━━━ 38s 28ms/step - accuracy: 0.8739 - loss: 0.2969 - val_accuracy: 0.8392 - val_loss: 0.3606


In [ ]:
# THIS BLOCK IS FOR GRADING
# DO NOT MODIFY THIS BLOCK

eval = model.evaluate(test_padded, test_labels)
q1_answer = [eval]
print(q1_answer)

782/782 ━━━━━━━━━━━━━━━━━━━━ 6s 8ms/step - accuracy: 0.8384 - loss: 0.3636
[[0.3606431782245636, 0.8392000198364258]]




---


# 02. Complete the Assignment


---



In [ ]:
import pandas as pd

answers = [q1_answer]

print("I confirm that I did all the homework in this assignment by myself.\nI understand that if I am found to have cheated, I will forfeit my right to receive a certificate.")
confirmation = input("If you confirm the statement given above, type 'I confirm' to submit: ")
data = {
    "name": ["name", "datetime", "confirmation", "answers"],
    "data": [name, datetimenow, confirmation, answers]
}
answers_df = pd.DataFrame(data)
answers_df.set_index("name", inplace=True)
answers_df.to_csv("w8_submission.csv", index=True)
model.save("model.h5")

I confirm that I did all the homework in this assignment by myself.
I understand that if I am found to have cheated, I will forfeit my right to receive a certificate.
If you confirm the statement given above, type 'I confirm' to submit: I confirm
